In [1]:
import yaml
import os

### Paths

In [2]:
OUTPUT_FILE = 'master/code/send_Alan/cfg_gen'       # Path of output of the generation

### Class for the cfg file

In [4]:
class Normalization:
    def __init__(self):

        self.normalization_type = 'default'

        self.position = {'minPos': -120, 'maxPos': 120}


        self.distance = {'minDistance': 0, 'maxDistance': 6}

        self.speed = {'minSpeed': -15, 'maxSpeed': 15}
        
        self.radius = {'minRad': 0, 'maxRad': 2}


    def to_dict(self):
        return self.__dict__


#########################

class Feature:
    def __init__(self):
        self.distGraph = 6
        self.nbHist = 4
        self.nbRolloutOut = 10
        self.output = 'speed'
        self.inShape = 8
        self.edgeShape = 5

    def to_dict(self):
        return self.__dict__


###########################

class SimulationParameters:
    def __init__(self):
        self.noisy = 0
        self.nMin = 120
        self.nMax = 300
        self.v0 = 60
        self.k = 70
        self.epsilon = 0.5
        self.tau = 3.5
        self.T = 1000
        self.dt = 0.001
        self.threshold = 6
        self.R = 1
        self.boundary = 120


    def to_dict(self):
        return self.__dict__

class Simulation:
    def __init__(self):
        self.nbSimLearning = 1000
        self.nbValidation = 20
        self.nbTest = 10
        self.initialization = 'easy'        # random, easy, circle
        self.initDistance = 7
        self.parameters = SimulationParameters()


    def to_dict(self):
        data = self.__dict__.copy()
        data['parameters'] = self.parameters.to_dict()
        return data


#############################

class DataAugment:
    def __init__(self):
        self.bool = 1
        self.prob = 0.8
        self.stdDeltaPos = 2
        self.stdSpeed = 0.003

    def to_dict(self):
        return self.__dict__

class Loss:
    def __init__(self):
        self.topk = -1
        self.lossScaling = 100
        self.l1Reg = 0.001
        self.lim = 35
        self.lambdaL2Weights = 0.00005


    def to_dict(self):
        return self.__dict__

class SchedulerExp:
    def __init__(self):
        self.gamma = 0.9

    def to_dict(self):
        return self.__dict__

class SchedulerLinear:
    def __init__(self):
        self.size = 1
        self.gamma = 0.5

    def to_dict(self):
        return self.__dict__

class Training:
    def __init__(self):
        self.nbEpoch = 3000
        self.modelName = 'simplest_dropout_no-encoder'
        self.evalModel = 'simplest_drop_no-enc_aug_best.pt'
        self.saveModel = 'simplest_drop_no-enc_aug_latest.pt'
        self.pathData = '/scratch/users/jpierre/mew_0.01_normal_v2'
        self.pathJsonBool = True
        self.pJson = ''
        self.wbName = 'Simplest_normal_0.001-lr_0_005_b-32'
        self.rolloutNb = 4
        self.batch = 16
        self.batchSizeVal = 128
        self.lr = 0.005
        self.dataAugment = DataAugment()
        self.loss = Loss()
        self.scheduler = 'exp'
        self.scheduler_exp = SchedulerExp()
        self.scheduler_linear = SchedulerLinear()
        self.frequenceEval = 50
        self.frequenceSave = 5000
        self.dt_update = 1
        self.tag = []
        self.training_type = '1-step'

    def to_dict(self):
        data = self.__dict__.copy()
        data['dataAugment'] = self.dataAugment.to_dict()
        data['loss'] = self.loss.to_dict()
        data['scheduler_exp'] = self.scheduler_exp.to_dict()
        data['scheduler_linear'] = self.scheduler_linear.to_dict()
        return data

class Config:
    def __init__(self):
        self.normalization = Normalization()
        self.feature = Feature()
        self.simulation = Simulation()
        self.training = Training()

    def to_dict(self):
        return {
            'normalization': self.normalization.to_dict(),
            'feature': self.feature.to_dict(),
            'simulation': self.simulation.to_dict(),
            'training': self.training.to_dict()
        }


def createYaml(data, f):

    with open(f'{f}.yml', 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

import os
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

### Create experiment settings

In [4]:
def createExpReLU():

    for dataType in ['normal']:
        for dt in [0.01]:
            for lr in [0.005]:
                for batch in [16]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['dropout', 'no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['update-data-exp_relu-exp']:
                                    for scaleL1 in [0.1, 0.01, 0.001, 0.0001, 0.00001]:
                                
                                
                                        wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_encoder-{encBool}_scaleL1-{scaleL1}_dropout-{dropBool}_{additional}'

                                        cfg = Config()
                                        cfg.training.batch = batch
                                        cfg.training.lr = lr
                                        cfg.training.wbName = wb_name

                                        modelName = 'simplest'
                                        modelName = modelName + f'_{dropBool}'
                                        modelName = modelName + f'_{encBool}'

                                        if 'relu-exp' in additional:
                                            modelName = modelName + '-relu'

                                        cfg.training.modelName = modelName

                                        cfg.training.loss.l1Reg = scaleL1


                                        if path_data in ['classic']:
                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                        elif path_data in ['simple']:
                                            cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal_v2'

                                        new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                        makedir(new_folder)
                                        
                                        createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [ ]:
createExpReLU()

In [13]:
def createBaseline():

    for dataType in ['normal', 'noisy']:
        for dt in [0.001]:
            for lr in [0.001]:
                for batch in [32]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['end-baseline', 'end-baseline-action']:
                                    for dataUpdate in ['v', 'delta']:
                                        for tr in ['rollout']:
                                            for scaleL1 in [0]:
                                    
                                                
                                                wb_name = f'mt_{dataType}_dType-{dataUpdate}_dt-{dt}_lr-{lr}-l1_{scaleL1}_batch-{batch}_tr-{tr}_{additional}'

                                                cfg = Config()
                                                cfg.training.batch = batch
                                                cfg.training.lr = lr
                                                cfg.training.wbName = wb_name
                                                cfg.training.training_type = tr

                                                if 'baseline' in additional:
                                                    modelName = 'baseline'
                                                    cfg.training.tag.append('baseline')
                                                else:
                                                    modelName = 'simplest'


                                                
                                                modelName = modelName + f'_{dropBool}'
                                                modelName = modelName + f'_{encBool}'

                                                if 'action' in additional:
                                                    cfg.training.tag.append('action-reaction')
                                                    modelName = modelName + '_action-reaction'

                                                #if 'relu-exp' in additional:
                                                #    modelName = modelName + '-relu'

                                                cfg.training.modelName = modelName
                                                cfg.training.saveModel = modelName + '_latest.pt'
                                                cfg.training.evalModel = modelName + '_best.pt'

                                                cfg.training.loss.l1Reg = scaleL1

                                                cfg.training.tag.append(dataUpdate)
                                                cfg.training.tag.append(f'dt-{dt}')
                                                cfg.training.tag.append(dataType)

                                                if path_data in ['classic']:
                                                    if dt == 0.001:
                                                        print('oo')
                                                        if dataUpdate == 'delta':
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                        elif dataUpdate == 'v':
                                                            cfg.training.dt_update = dt
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                    elif dt == 0.01:
                                                        if dataUpdate == 'delta':
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                        elif dataUpdate == 'v':
                                                            cfg.training.dt_update = dt
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                                makedir(new_folder)
                                                
                                                createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))

                                                cfg.training.tag = []


In [14]:
createBaseline()

oo
oo
oo
oo
oo
oo
oo
oo


In [24]:
def createGat():

    for dataType in ['normal', 'noisy']:
        for dt in [0.01, 0.001]:
            for lr in [0.001]:
                for batch in [32]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['end-gat']:
                                    for dataUpdate in ['v', 'delta']:
                                        for n_layers in [3]:
                                            for scaleL1 in [0]:
                                    
                                    
                                                wb_name = f'mt_{dataType}_dType-{dataUpdate}_dt-{dt}_lr-{lr}-nlayers_{n_layers}l1_{scaleL1}_batch-{batch}_{additional}'

                                                cfg = Config()
                                                cfg.training.batch = batch
                                                cfg.training.lr = lr
                                                cfg.training.wbName = wb_name

                                                if 'gat' in additional:
                                                    modelName = 'gat'
                                                    cfg.training.tag.append('gat')
                                                    cfg.training.tag.append(f'n-layers_{3}')
                                                else:
                                                    modelName = 'simplest'
                                                
                                                modelName = modelName + f'_{dropBool}'
                                                modelName = modelName + f'_{encBool}'

                                                #if 'relu-exp' in additional:
                                                #    modelName = modelName + '-relu'

                                                cfg.training.modelName = modelName
                                                cfg.training.saveModel = modelName + '_latest.pt'
                                                cfg.training.evalModel = modelName + '_best.pt'

                                                cfg.training.loss.l1Reg = scaleL1

                                                cfg.training.tag.append(dataUpdate)
                                                cfg.training.tag.append(f'dt-{dt}')
                                                cfg.training.tag.append(dataType)

                                                if path_data in ['classic']:
                                                    if dt == 0.001:
                                                        print('oo')
                                                        if dataUpdate == 'delta':
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                        elif dataUpdate == 'v':
                                                            cfg.training.dt_update = dt
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                    elif dt == 0.01:
                                                        if dataUpdate == 'delta':
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                        elif dataUpdate == 'v':
                                                            cfg.training.dt_update = dt
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                                makedir(new_folder)
                                                
                                                createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [25]:
createGat()

oo
oo
oo
oo


In [27]:
def createCompex():

    for dataType in ['normal', 'noisy']:
        for dt in [0.01, 0.001]:
            for lr in [0.001]:
                for batch in [32]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['end-compex-gnn']:
                                    for dataUpdate in ['v', 'delta']:
                                        for n_layers in [3]:
                                            for scaleL1 in [0]:
                                    
                                    
                                                wb_name = f'mt_{dataType}_dType-{dataUpdate}_dt-{dt}_lr-{lr}-nlayers_{n_layers}l1_{scaleL1}_batch-{batch}_{additional}'

                                                cfg = Config()
                                                cfg.training.batch = batch
                                                cfg.training.lr = lr
                                                cfg.training.wbName = wb_name

                                                if 'compex-gnn' in additional:
                                                    modelName = 'compex-gnn'
                                                    cfg.training.tag.append('compex-gnn')
                                                    cfg.training.tag.append(f'n-layers_{3}')
                                                else:
                                                    modelName = 'simplest'
                                                
                                                modelName = modelName + f'_{dropBool}'
                                                modelName = modelName + f'_{encBool}'

                                                #if 'relu-exp' in additional:
                                                #    modelName = modelName + '-relu'

                                                cfg.training.modelName = modelName
                                                cfg.training.saveModel = modelName + '_latest.pt'
                                                cfg.training.evalModel = modelName + '_best.pt'

                                                cfg.training.loss.l1Reg = scaleL1

                                                cfg.training.tag.append(dataUpdate)
                                                cfg.training.tag.append(f'dt-{dt}')
                                                cfg.training.tag.append(dataType)

                                                if path_data in ['classic']:
                                                    if dt == 0.001:
                                                        print('oo')
                                                        if dataUpdate == 'delta':
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                        elif dataUpdate == 'v':
                                                            cfg.training.dt_update = dt
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                    elif dt == 0.01:
                                                        if dataUpdate == 'delta':
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                        elif dataUpdate == 'v':
                                                            cfg.training.dt_update = dt
                                                            if dataType == 'normal':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_normal_v2'
                                                            elif dataType == 'noisy':
                                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_noisy_v2'
                                                            else:
                                                                print('ISSUE NORMAL-NOISY')

                                                new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                                makedir(new_folder)
                                                
                                                createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [28]:
createCompex()

oo
oo
oo
oo


In [16]:
def createbatchlr():

    for dataType in ['normal', 'noisy']:
        for dt in [0.01, 0.001]:
            for lr in [0.001]:
                for batch in [32]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['end']:
                                    for dataUpdate in ['v', 'delta']:
                                        for scaleL1 in [0.01, 0.001, 0.0001, 0.00001]:
                                    
                                    
                                            wb_name = f'mt_{dataType}_dType-{dataUpdate}_dt-{dt}_lr-{lr}-l1_{scaleL1}_batch-{batch}_{additional}'

                                            cfg = Config()
                                            cfg.training.batch = batch
                                            cfg.training.lr = lr
                                            cfg.training.wbName = wb_name

                                            if 'basline-model' in additional:
                                                modelName = 'baseline'
                                            else:
                                                modelName = 'simplest'
                                            
                                            modelName = modelName + f'_{dropBool}'
                                            modelName = modelName + f'_{encBool}'

                                            if 'relu-exp' in additional:
                                                modelName = modelName + '-relu'

                                            cfg.training.modelName = modelName
                                            cfg.training.saveModel = modelName + '_latest.pt'
                                            cfg.training.evalModel = modelName + '_best.pt'

                                            cfg.training.loss.l1Reg = scaleL1


                                            if path_data in ['classic']:
                                                if dt == 0.001:
                                                    print('oo')
                                                    if dataUpdate == 'delta':
                                                        if dataType == 'normal':
                                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                        elif dataType == 'noisy':
                                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                        else:
                                                            print('ISSUE NORMAL-NOISY')

                                                    elif dataUpdate == 'v':
                                                        cfg.training.dt_update = dt
                                                        if dataType == 'normal':
                                                            cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_normal_v2'
                                                        elif dataType == 'noisy':
                                                            cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_noisy_v2'
                                                        else:
                                                            print('ISSUE NORMAL-NOISY')

                                                elif dt == 0.01:
                                                    if dataUpdate == 'delta':
                                                        if dataType == 'normal':
                                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_normal_v2'
                                                        elif dataType == 'noisy':
                                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_noisy_v2'
                                                        else:
                                                            print('ISSUE NORMAL-NOISY')

                                                    elif dataUpdate == 'v':
                                                        cfg.training.dt_update = dt
                                                        if dataType == 'normal':
                                                            cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_normal_v2'
                                                        elif dataType == 'noisy':
                                                            cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_noisy_v2'
                                                        else:
                                                            print('ISSUE NORMAL-NOISY')
                                                    
                                            elif path_data in ['simple']:
                                                cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal_v2'

                                            new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                            makedir(new_folder)
                                            
                                            createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [17]:
createbatchlr()

oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo


In [13]:
def symbolicDistillation1():

    for dataType in ['normal']:
        for dt in [0.001]:
            for lr in [0.05, 0.005, 0.0005]:
                for batch in [16, 32, 64]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['symbolic_force']:
                                    for scaleL1 in [0.001]:
                                
                                
                                        wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_encoder-{encBool}__dropout-{dropBool}_{additional}'

                                        cfg = Config()
                                        cfg.training.batch = batch
                                        cfg.training.lr = lr
                                        cfg.training.wbName = wb_name

                                        if 'symbolic_force' in additional:
                                            modelName = 'symbolic_GNN'
                                        

                                        cfg.training.modelName = modelName
                                        cfg.training.saveModel = modelName + '_latest.pt'
                                        cfg.training.evalModel = modelName + '_best.pt'

                                        cfg.training.loss.l1Reg = scaleL1


                                        if path_data in ['classic']:
                                            if dt == 0.001:
                                                print('oo')
                                                if dataType == 'normal':
                                                    cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                elif dataType == 'noisy':
                                                    cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                else:
                                                    print('ISSUE NORMAL-NOISY')
                                        elif path_data in ['simple']:
                                            cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal_v2'

                                        new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                        makedir(new_folder)
                                        
                                        createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [17]:
symbolicDistillation1()

oo
oo
oo
oo
oo
oo
oo
oo
oo


In [ ]:
def symbolicDistillation2():

    for dataType in ['normal']:
        for dt in [0.001]:
            for lr in [0.05, 0.005, 0.0005]:
                for batch in [8, 32, 64]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['batch-lr-exp', 'batch-lr-relu-exp']:
                                    for scaleL1 in [0.001]:
                                
                                
                                        wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_encoder-{encBool}__dropout-{dropBool}_{additional}'

                                        cfg = Config()
                                        cfg.training.batch = batch
                                        cfg.training.lr = lr
                                        cfg.training.wbName = wb_name

                                        if 'basline-model' in additional:
                                            modelName = 'baseline'
                                        else:
                                            modelName = 'simplest'
                                        
                                        modelName = modelName + f'_{dropBool}'
                                        modelName = modelName + f'_{encBool}'

                                        if 'relu-exp' in additional:
                                            modelName = modelName + '-relu'

                                        cfg.training.modelName = modelName
                                        cfg.training.saveModel = modelName + '_latest.pt'
                                        cfg.training.evalModel = modelName + '_best.pt'

                                        cfg.training.loss.l1Reg = scaleL1


                                        if path_data in ['classic']:
                                            if dt == 0.001:
                                                print('oo')
                                                if dataType == 'normal':
                                                    cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                elif dataType == 'noisy':
                                                    cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                else:
                                                    print('ISSUE NORMAL-NOISY')
                                        elif path_data in ['simple']:
                                            cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal_v2'

                                        new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                        makedir(new_folder)
                                        
                                        createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [ ]:
symbolicDistillation2()

In [ ]:
def GAM():

    for dataType in ['normal']:
        for dt in [0.001]:
            for lr in [0.05, 0.005, 0.0005]:
                for batch in [8, 32, 64]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['batch-lr-exp', 'batch-lr-relu-exp']:
                                    for scaleL1 in [0.001]:
                                
                                
                                        wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_encoder-{encBool}__dropout-{dropBool}_{additional}'

                                        cfg = Config()
                                        cfg.training.batch = batch
                                        cfg.training.lr = lr
                                        cfg.training.wbName = wb_name

                                        if 'basline-model' in additional:
                                            modelName = 'baseline'
                                        else:
                                            modelName = 'simplest'
                                        
                                        modelName = modelName + f'_{dropBool}'
                                        modelName = modelName + f'_{encBool}'

                                        if 'relu-exp' in additional:
                                            modelName = modelName + '-relu'

                                        cfg.training.modelName = modelName
                                        cfg.training.saveModel = modelName + '_latest.pt'
                                        cfg.training.evalModel = modelName + '_best.pt'

                                        cfg.training.loss.l1Reg = scaleL1


                                        if path_data in ['classic']:
                                            if dt == 0.001:
                                                print('oo')
                                                if dataType == 'normal':
                                                    cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                                elif dataType == 'noisy':
                                                    cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                                else:
                                                    print('ISSUE NORMAL-NOISY')
                                        elif path_data in ['simple']:
                                            cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal_v2'

                                        new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                        makedir(new_folder)
                                        
                                        createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [23]:
def createGAM():

    for dataType in ['normal', 'noisy']:
        for dt in [0.001]:
            for lr in [0.001]:
                for batch in [32]:
                    for encBool in ['no-encoder']:
                        for dropBool in ['no-dropout']:
                            for path_data in ['classic']:
                                for additional in ['end-gam-action']:
                                    for dataUpdate in ['v', 'delta']:
                                        for scaleL1 in [1, 0.00001]:
                        
                        
                                wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_l1scale-{scaleL1}_{additional}'

                                cfg = Config()
                                cfg.training.batch = batch
                                cfg.training.lr = lr
                                cfg.training.wbName = wb_name

                                modelName = 'GAM_GNN'
                                

                                if 'relu-exp' in additional:
                                    modelName = modelName + '-relu'

                                cfg.training.modelName = modelName
                                cfg.training.saveModel = modelName + '_latest.pt'
                                cfg.training.evalModel = modelName + '_best.pt'

                                cfg.training.loss.l1Reg = scaleL1


                                cfg.training.tag.append(dataUpdate)
                                cfg.training.tag.append(f'dt-{dt}')
                                cfg.training.tag.append(dataType)

                                if path_data in ['classic']:
                                    if dt == 0.001:
                                        print('oo')
                                        if dataUpdate == 'delta':
                                            if dataType == 'normal':
                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                            elif dataType == 'noisy':
                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                            else:
                                                print('ISSUE NORMAL-NOISY')

                                        elif dataUpdate == 'v':
                                            cfg.training.dt_update = dt
                                            if dataType == 'normal':
                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_normal_v2'
                                            elif dataType == 'noisy':
                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.001_noisy_v2'
                                            else:
                                                print('ISSUE NORMAL-NOISY')

                                    elif dt == 0.01:
                                        if dataUpdate == 'delta':
                                            if dataType == 'normal':
                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_normal_v2'
                                            elif dataType == 'noisy':
                                                cfg.training.pathData = '/scratch/users/jpierre/mew_0.01_noisy_v2'
                                            else:
                                                print('ISSUE NORMAL-NOISY')

                                        elif dataUpdate == 'v':
                                            cfg.training.dt_update = dt
                                            if dataType == 'normal':
                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_normal_v2'
                                            elif dataType == 'noisy':
                                                cfg.training.pathData = '/scratch/users/jpierre/aquali_speed_upd/aquali_0.01_noisy_v2'
                                            else:
                                                print('ISSUE NORMAL-NOISY')

                                new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                makedir(new_folder)
                                
                                createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


                                cfg.training.tag = []


In [24]:
createGAM()

oo
oo
oo
oo
oo
oo


In [27]:
def createGAM_relu():

    for dataType in ['normal']:
        for dt in [0.001]:
            for lr in [0.05, 0.005, 0.0005]:
                for batch in [16]:

                    for path_data in ['classic']:
                        for additional in ['gam_test_scale-relu-exp']:
                            for scaleL1 in [100, 10, 1, 0.1, 0.01, 0.001]:
                        
                        
                                wb_name = f'master-thesis_{dataType}_{path_data}_dt-{dt}_lr-{lr}_batch-{batch}_l1scale-{scaleL1}_{additional}'

                                cfg = Config()
                                cfg.training.batch = batch
                                cfg.training.lr = lr
                                cfg.training.wbName = wb_name

                                modelName = 'GAM_GNN'
                                

                                if 'relu-exp' in additional:
                                    modelName = modelName + '-relu'

                                cfg.training.modelName = modelName
                                cfg.training.saveModel = modelName + '_latest.pt'
                                cfg.training.evalModel = modelName + '_best.pt'

                                cfg.training.loss.l1Reg = scaleL1


                                if path_data in ['classic']:
                                    if dt == 0.001:
                                        print('oo')
                                        if dataType == 'normal':
                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_normal_v2'
                                        elif dataType == 'noisy':
                                            cfg.training.pathData = '/scratch/users/jpierre/mew_0.001_noisy_v2'
                                        else:
                                            print('ISSUE NORMAL-NOISY')
                                elif path_data in ['simple']:
                                    cfg.training.pathData = '/scratch/users/jpierre/smallest_0.001_normal_v2'

                                new_folder = os.path.join(OUTPUT_FILE, wb_name)
                                makedir(new_folder)
                                
                                createYaml(cfg.to_dict(), os.path.join(new_folder, 'cfg'))


In [28]:
createGAM_relu()

oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo
oo


### Run those experiments

In [20]:
createExpReLU()

### Get the commands (useless now)

In [37]:
def getCommands(Alan_nb = 2):
    cd_command = f'cd /home/jpierre/v2/new_runs/Alan/Alan{Alan_nb}/ \n'
    conda_command = 'conda activate myenvPy \n'
    py_command = 'python training_run.py'

    print(cd_command + conda_command + py_command)

In [38]:
getCommands()

cd /home/jpierre/v2/new_runs/Alan/Alan2/ 
conda activate myenvPy 
python training_run.py


In [45]:
import numpy as np
a = np.array([[[1, 2]], [[3, 4]]])
b = np.array([[1, 2], [3, 4]])

In [46]:
print((a-b))

[[[ 0  0]
  [-2 -2]]

 [[ 2  2]
  [ 0  0]]]
